# Mean Reversion Trading of Calendar Spreads

## Imports

In [1]:
import pandas as pd
import numpy as np
from statsmodels.api import OLS
import statsmodels.api as sm

## Load Data

In [2]:
F_CLdf = pd.read_csv('data/F_CL_history.csv', skiprows=1)
F_CLdf.columns = ['Date', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change %']
F_CLdf['Date'] = pd.to_datetime(F_CLdf['Date'], errors='coerce')
F_CLdf['Price'] = pd.to_numeric(F_CLdf['Price'], errors='coerce')

## Logic

In [3]:
contracts = [F_CLdf[i:i+252] for i in range(0, len(F_CLdf), 252)]

positions = np.zeros_like(F_CLdf['Price'])
spreadMonth = 12
holddays = 3*21
numDaysStart = holddays + 10
numDaysEnd = 10

for i in range(len(contracts) - spreadMonth):
    near_contract = contracts[i]
    far_contract = contracts[i + spreadMonth]
    
    # Dummy expiration date as the last date of the contract data
    expiration_date = near_contract['Date'].iloc[-1]
    
    log_spread = np.log(far_contract['Price']) - np.log(near_contract['Price'])
    gamma = log_spread  

    gamma_lagged = gamma.shift(1).dropna()
    delta_gamma = gamma - gamma_lagged
    regress_results = OLS(delta_gamma[1:], sm.add_constant(gamma_lagged)).fit()
    halflife = -np.log(2) / regress_results.params[0]

    lookback = round(halflife)
    ma = gamma.rolling(lookback).mean()
    mstd = gamma.rolling(lookback).std()
    z_score = (gamma - ma) / mstd
    
    start_idx = max(0, (expiration_date - F_CLdf['Date'][0]).days - numDaysStart)
    end_idx = (expiration_date - F_CLdf['Date'][0]).days - numDaysEnd
    positions[start_idx:end_idx] = np.where(z_score[start_idx:end_idx] > 0, -1, 1)

    if i >= len(contracts) - spreadMonth - 5:  # For the last 5 iterations
        print("Gamma:\n", gamma.tail())
        print("MA:\n", ma.tail())
        print("MSTD:\n", mstd.tail())
        print("Z-Score:\n", z_score.tail())
        print("Start and End Indices:", start_idx, end_idx)

## Returns and Statistics

In [4]:
positions_shifted = np.roll(positions, 1, axis=0)
positions_shifted[0] = 0 

F_CLdf['Returns'] = F_CLdf['Price'].pct_change()
strategy_returns = F_CLdf['Returns'] * positions_shifted
cumulative_returns = (1 + strategy_returns).cumprod() - 1

APR = (cumulative_returns.iloc[-1] + 1)**(252 / len(cumulative_returns)) - 1
print(f"APR: {APR*100:.2f}%")

risk_free_rate = 0.053
mean_return = strategy_returns.mean() * 252
volatility = strategy_returns.std() * np.sqrt(252)
sharpe_ratio = (mean_return - risk_free_rate) / volatility
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

print(f"Last Cumulative Return: {cumulative_returns.iloc[-1]:.4f}")
print(f"Number of Non-Zero Strategy Returns: {(strategy_returns != 0).sum()}")
print(f"Number of NaN in Strategy Returns: {strategy_returns.isna().sum()}")
print(f"Volatility: {volatility:.4f}")


APR: 0.00%
Sharpe Ratio: -inf
Last Cumulative Return: 0.0000
Number of Non-Zero Strategy Returns: 1
Number of NaN in Strategy Returns: 1
Volatility: 0.0000


C:\Users\Arjun\AppData\Local\Temp\ipykernel_26072\1700089478.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (mean_return - risk_free_rate) / volatility
